In [19]:
!pip install langchain langchain-community langchain-huggingface faiss-cpu pypdf2 python-dotenv groq

In [20]:
from google.colab import userdata
import os
from dotenv import load_dotenv

# Store your Groq API key in Colab's secrets (userdata)
# Go to the "Secrets" tab in Colab (lock icon), add key 'GROQ_API_KEY' with your API key
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

# Verify the API key is set
print("GROQ_API_KEY set successfully!" if os.environ.get("GROQ_API_KEY") else "GROQ_API_KEY not set.")

GROQ_API_KEY set successfully!


In [21]:
from google.colab import files
import PyPDF2

# Upload the car manual PDF
uploaded = files.upload()

# Assuming you uploaded a file named 'car_manual.pdf'
pdf_path = list(uploaded.keys())[0]

# Read and extract text from the PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

pdf_text = extract_text_from_pdf(pdf_path)
print(f"Extracted {len(pdf_text)} characters from the PDF.")

Saving 2025_MustangS650_OM_ENG_version1.pdf to 2025_MustangS650_OM_ENG_version1 (1).pdf
Extracted 927725 characters from the PDF.


In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Max size of each chunk
    chunk_overlap=150  # Overlap to maintain context
)

# Split the text into chunks
chunks = text_splitter.split_text(pdf_text)
print(f"Created {len(chunks)} document chunks.")

Created 1089 document chunks.


In [23]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document

# Initialize the embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert chunks to LangChain Document objects
documents = [Document(page_content=chunk) for chunk in chunks]

# Create a FAISS vector store
vector_store = FAISS.from_documents(documents, embeddings)
print("FAISS vector store created successfully.")

FAISS vector store created successfully.


In [24]:
!pip install langchain-groq

In [25]:
from langchain_groq import ChatGroq

# Initialize the Groq LLM
llm = ChatGroq(
    model_name="llama3-8b-8192",  # Changed to a currently supported model
    temperature=0.7  # Controls randomness (0.7 is a good balance)
)
print("Groq LLM initialized.")

Groq LLM initialized.


In [26]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Define a custom prompt template
prompt_template = """You are a helpful car assistant. Use the following context
                      to answer the user's question about dashboard warnings or driving recommendations.
                      Provide a clear, concise, and accurate response based on the car manual.
                      If you don't know the answer, say so.

Context: {context}

Question: {question}

Answer:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Create the RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Uses all retrieved documents directly
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),  # Retrieve top 3 relevant chunks
    chain_type_kwargs={"prompt": prompt}
)
print("RAG pipeline created.")

RAG pipeline created.


In [27]:
from IPython.display import display, Markdown
import ipywidgets as widgets

# Create input and output widgets
input_box = widgets.Text(placeholder="Enter your question about the car manual", description="Question:")
output_area = widgets.Output()
button = widgets.Button(description="Ask")

# Define button click handler
def on_button_clicked(b):
    with output_area:
        output_area.clear_output()
        question = input_box.value
        if question:
            response = rag_chain.run(question)
            display(Markdown(f"**Question**: {question}\n\n**Answer**: {response}"))
        else:
            display(Markdown("Please enter a question."))

button.on_click(on_button_clicked)

# Display the interface
display(input_box)
display(button)
display(output_area)

Text(value='', description='Question:', placeholder='Enter your question about the car manual')

Button(description='Ask', style=ButtonStyle())

Output()